# 4. Extraer datos de cada partido

##   --   4.1 ruta completa de los archivos

In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tqdm

In [22]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

El listado de nombres de archivos almacenados en `score_dir` está desactualizado. Una vez descargada la totalidad de archivos (aproximadamente el domingo) tendré que correr de nuevo este código.

In [23]:
scores_dir = "datanba/scores"

In [24]:
box_scores = os.listdir(scores_dir)

In [25]:
len(box_scores)

12836

In [26]:
# confirmar que sólo hay elementos finalizados con .html

contador = 0
for b in box_scores:
    if b.endswith(".html"):
        contador += 1
print(f"EL CONTADOR ES IGUAL A: {contador}")

EL CONTADOR ES IGUAL A: 12836


In [27]:
# ruta completa:

box_scores = [os.path.join(scores_dir, box).replace('\\', '/') for box in box_scores]

In [28]:
box_scores[0:4]

['datanba/scores/201310290IND.html',
 'datanba/scores/201310290LAL.html',
 'datanba/scores/201310290MIA.html',
 'datanba/scores/201310300CLE.html']

----------------------------------------------------------------------------------------------------

## -- 4.3 Eliminar encabezados y comentarios de las tablas
Después de establecer la ruta completa, es necesario quitar los encabezados de las tablas. Para eso crearé una función en la que elimine las etiquetas que incluyan `<tr class="over_header">` y `<tr class="thead" ...>`

Quiero las tablas sin cabeceras y sin fila intermedia (con variables), así que procedo a eliminar esas líneas mediante sus etiquetas, pero antes debo descomentar las líneas de código donde está la tabla **line scoring**.

In [29]:
def get_clean_html(box_score):
    with open(box_score, 'r', encoding='utf-8') as b:
        html = b.read()
    html_soup = BeautifulSoup(html, 'html.parser')

    html_soup = str(html_soup).replace("<!--", "").replace("-->","")
    soup = BeautifulSoup(html_soup, 'html.parser')
    
    [x.decompose() for x in soup.select("tr.over_header")] #elimina la fila de la cabecera de la tabla
    [x.decompose() for x in soup.select("tr.thead")] #elimina la fila intermedia en las tablas basic y advanced box score stats

    return soup


## -- 4.2 Extraer la tabla Line Scoring:

Solo tomo el resultado final y no los puntos realizados en cada tiempo porque el partido no puede finalizar con empate y tienen que jugar prórrogas de 5 minutos hasta que se resuelva el empate, así que en cada partido puede haber diferentes tiempos en total si se cuentan las prórrogas.

In [30]:
def line_scoring_table(soup):
    line_scoring = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_scoring.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_scoring.columns = cols
    
    line_scoring = line_scoring[["team", "total"]]
    teams = list(line_scoring["team"])
    return line_scoring

## -- 4.3 Extraer las tablas Basic y Advanced Box Score Stats

Estas tablas muestran distintas estadísticas de los jugadores de cada equipo. Se diferencia entre estadísticas básicas y avanzadas, y a su vez, se divide en equipos, por tanto, hay cuatro tablas por partido (sin incluir la tabla obtenida anteriomente).

Al observar el glosario de variables (https://www.basketball-reference.com/about/glossary.html), llego a la conclusión que todas las variables existen desde el momento en que se toman los datos, sin ser necesario hacer eliminación de variables (de todos modos, depende de cómo vaya de tiempo puedo hacer un bucle para mirar si hay alguna variable nueva que no se repita o se repita varias veces).

In [31]:
def basic_adv_tables(soup, team, stat):
    df = pd.read_html(str(soup), attrs={'id':f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

## -- 4.4 Seleccionar temporada y fecha

In [32]:
def game_season(soup):
    id_date = soup.find('div', class_='box').find('h1').text
    match = re.search(r'\b\d{4}\b', id_date)
    year = match.group()
    return year

## -- 4.5 Agregar seleccionar variables

Las tablas `basic` y `adv` tienen valores nulos, así que hay que quitarlos.

Elección de filas y tipos de valores:
 - fila: Team Totals ( valor en código: **team_totals**)
 - valores máximos: **max_value**
 
En total, habrá el doble de filas que partidos disputados porque el propósito es mostrar el desempeño de un equipo frente al oponente en un partido en concreto, saber qué estadísticas hay cuando juega contra ese otro equipo.

- variable ``"home"``: 0=away team, 1=home team
- variable ``"winner"``: True: gana el equipo de la variable "team", Falso: pierde el equipo de la variable "team".

##### Código inicial con el que se obtienen las tablas: line scoring, basic y advanced stats. 

En el resultado de este código hay 51 errores, así que tengo que modificar el código para saber qué error da cada archivo .html

In [34]:
# ORIGINAL

ref_cols = None
full_matchup = []
count = 0
corrupted_box = []

for boxscore in tqdm(box_scores):
    game_file_name = os.path.basename(boxscore)
    
    if game_file_name in box_scores:
        continue
    
    try:
        soup = get_clean_html(boxscore)
        line_scoring = line_scoring_table(soup)
        teams = list(line_scoring["team"])


        mixes = []
        for team in teams:
            basic = basic_adv_tables(soup, team, "basic")
            adv = basic_adv_tables(soup, team, "advanced")
            
            if 'Unnamed: 16' in basic.columns:
                basic = basic.drop(columns=['Unnamed: 16'])
            
            if 'Unnamed: 16' in adv.columns:
                adv = adv.drop(columns=['Unnamed: 16'])

            team_totals = pd.concat([basic.iloc[-1,:], adv.iloc[-1,:]])

            max_value = pd.concat([basic.iloc[:-1], adv.iloc[:-1]]).max()

            max_value.index = max_value.index + "_max"

            mix = pd.concat([team_totals, max_value])

            #marca las variables de referencia
            if ref_cols is None:
                ref_cols = list(mix.index.drop_duplicates(keep="first"))
                # Arregla la ausencia de la variable BMP
                ref_cols = [x for x in ref_cols if "BMP" not in x]

            mix = mix[ref_cols]

            mixes.append(mix)

        mix = pd.concat(mixes, axis = 1).T

        matchup = pd.concat([mix, line_scoring], axis = 1)
        matchup["home"] = [0, 1] #0=away team, 1=home team
        matchup_opponent = matchup.iloc[::-1].reset_index()
        matchup_opponent.columns = matchup_opponent.columns + "_opponent"

        full_matchup_stats = pd.concat([matchup, matchup_opponent], axis = 1)

        #Fecha
        full_matchup_stats["season"] = game_season(soup)
        full_matchup_stats["date"] = pd.to_datetime(os.path.basename(boxscore)[:8], format = "%Y%m%d")

        #Ganador partido. Dice si el equipo que aparece en "team" es el ganador o no
        full_matchup_stats["winner"] = full_matchup_stats["total"] > full_matchup_stats["total_opponent"]
        full_matchup.append(full_matchup_stats)
        
    except Exception as e:
        count += 1
        print(f"Number of corrupted box: {count}")
        print(f"Corrupted box: {game_file_name}")
        print(e)
        corrupted_box.append(game_file_name)
        
        continue
        


 70%|██████████████████████████████████████████████████▊                      | 8939/12836 [4:19:51<1:30:53,  1.40s/it]

Number of corrupted box: 1
Corrupted box: 202008150POR.html
"['BPM', 'BPM_max'] not in index"


 79%|████████████████████████████████████████████████████████▋               | 10103/12836 [4:53:26<1:11:22,  1.57s/it]

Number of corrupted box: 2
Corrupted box: 202105180BOS.html
"['BPM', 'BPM_max'] not in index"


 79%|████████████████████████████████████████████████████████▋               | 10104/12836 [4:53:27<1:09:58,  1.54s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████▌| 12751/12836 [6:47:02<01:59,  1.40s/it]

Number of corrupted box: 18
Corrupted box: 202304140MIA.html
"['BPM', 'BPM_max'] not in index"


 99%|█████████████████████████████████████████████████████████████████████████▌| 12752/12836 [6:47:03<01:50,  1.32s/it]

Number of corrupted box: 19
Corrupted box: 202304140MIN.html
"['BPM', 'BPM_max'] not in index"


100%|██████████████████████████████████████████████████████████████████████████| 12836/12836 [6:49:25<00:00,  1.91s/it]


In [35]:
full_matchup

[      MP     MP    FG   FGA    FG%   3P   3PA    3P%    FT   FTA  ...  \
 0  240.0  240.0  36.0  93.0  0.387  9.0  19.0  0.474   6.0  10.0  ...   
 1  240.0  240.0  34.0  71.0  0.479  7.0  17.0  0.412  22.0  32.0  ...   
 
    USG%_max_opponent  ORtg_max_opponent  DRtg_max_opponent  BPM_max_opponent  \
 0               29.2              150.0              104.0              10.1   
 1               30.1              151.0              111.0              17.3   
 
    team_opponent  total_opponent  home_opponent  season       date  winner  
 0            IND              97              1    2013 2013-10-29   False  
 1            ORL              87              0    2013 2013-10-29    True  
 
 [2 rows x 152 columns],
       MP     MP    FG   FGA    FG%    3P   3PA    3P%    FT   FTA  ...  \
 0  240.0  240.0  41.0  83.0  0.494   8.0  21.0  0.381  13.0  23.0  ...   
 1  240.0  240.0  42.0  93.0  0.452  14.0  29.0  0.483  18.0  28.0  ...   
 
    USG%_max_opponent  ORtg_max_opponent  D

Si no hay archivos corruptos, salta al punto 4.7





## -- 4.6 Corregir archivos corruptos

Se ejecutará si hay archivos corruptos

##### Aquí se guarda la lista ``corrupted_box`` en un archivo .txt, y se crea la variable ``box_scores_corrupted`` para obtener la ruta completa de cada archivo

In [36]:
# Guardar la lista en un archivo de texto:

file = "corruptedbox.txt"

with open(file, "w") as f:
    for x in corrupted_box:
        f.write(x + "\n")

##### Después de guardar la lista de errores y volver a descargarlo, se vuelve a tratar los archivos corruptos

Se hará en caso de haber parado la ejecución del notebook y se haya perdido el valor de la variable ``corrupted_box``

In [17]:
# Abrir el archivo de texto y guardarlo en una lista:

file = "corruptedbox.txt"
corrupted_box = []

with open(file, "r") as x:
    for l in x:
        corrupted_box.append(l.strip())

In [37]:
# ruta completa:

box_scores_corrupted = [os.path.join(scores_dir, box).replace('\\', '/') for box in corrupted_box]

In [38]:
box_scores_corrupted

['datanba/scores/202008150POR.html',
 'datanba/scores/202105180BOS.html',
 'datanba/scores/202105180IND.html',
 'datanba/scores/202105190LAL.html',
 'datanba/scores/202105190MEM.html',
 'datanba/scores/202105200WAS.html',
 'datanba/scores/202105210GSW.html',
 'datanba/scores/202204120BRK.html',
 'datanba/scores/202204120MIN.html',
 'datanba/scores/202204130ATL.html',
 'datanba/scores/202204130NOP.html',
 'datanba/scores/202204150CLE.html',
 'datanba/scores/202204150LAC.html',
 'datanba/scores/202304110LAL.html',
 'datanba/scores/202304110MIA.html',
 'datanba/scores/202304120NOP.html',
 'datanba/scores/202304120TOR.html',
 'datanba/scores/202304140MIA.html',
 'datanba/scores/202304140MIN.html']

##### Después de obtener la ruta completa del archivo, se corre el siguiente código. 
Consiste en arreglar el problema e imprimir el error si sigue existiendo.


In [66]:
# IMPRIMIR EL ERROR DEL ARCHIVO CORRUPTO

ref_cols = None
full_matchup_corrupted = []
count = 0
corrupted_box = []

for boxscore in tqdm(box_scores_corrupted):
    game_file_name = os.path.basename(boxscore)
    
    if game_file_name in box_scores_corrupted:
        continue
    
    try:
        soup = get_clean_html(boxscore)
        line_scoring = line_scoring_table(soup)
        teams = list(line_scoring["team"])


        mixes = []
        for team in teams:
            basic = basic_adv_tables(soup, team, "basic")
            adv = basic_adv_tables(soup, team, "advanced")
            adv['BPM'] = None
            
            if 'Unnamed: 16' in basic.columns:
                basic = basic.drop(columns=['Unnamed: 16'])
            
            if 'Unnamed: 16' in adv.columns:
                adv = adv.drop(columns=['Unnamed: 16'])

            team_totals = pd.concat([basic.iloc[-1,:], adv.iloc[-1,:]])

            max_value = pd.concat([basic.iloc[:-1], adv.iloc[:-1]]).max()

            max_value.index = max_value.index + "_max"

            mix = pd.concat([team_totals, max_value])

            #marca las variables de referencia
            if ref_cols is None:
                ref_cols = list(mix.index.drop_duplicates(keep="first"))

            mix = mix[ref_cols]

            mixes.append(mix)

        mix = pd.concat(mixes, axis = 1).T

        matchup = pd.concat([mix, line_scoring], axis = 1)
        matchup["home"] = [0, 1] #0=away team, 1=home team
        matchup_opponent = matchup.iloc[::-1].reset_index()
        matchup_opponent.columns = matchup_opponent.columns + "_opponent"

        full_matchup_stats = pd.concat([matchup, matchup_opponent], axis = 1)

        #Fecha
        full_matchup_stats["season"] = game_season(soup)
        full_matchup_stats["date"] = pd.to_datetime(os.path.basename(boxscore)[:8], format = "%Y%m%d")

        #Ganador partido. Dice si el equipo que aparece en "team" es el ganador o no
        full_matchup_stats["winner"] = full_matchup_stats["total"] > full_matchup_stats["total_opponent"]
        full_matchup_corrupted.append(full_matchup_stats)
        
    except Exception as e:
        count += 1
        print(f"Number of corrupted box: {count}")
        print(f"Corrupted box: {game_file_name}")
        print(e)
        
        continue      

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:30<00:00,  1.62s/it]


In [67]:
full_matchup_corrupted

[      MP     MP    FG   FGA    FG%    3P   3PA    3P%    FT   FTA    FT%  \
 0  240.0  240.0  45.0  92.0  0.489  14.0  25.0   0.56  18.0  22.0  0.818   
 1  240.0  240.0  38.0  84.0  0.452  15.0  40.0  0.375  35.0  41.0  0.854   
 
     ORB   DRB   TRB   AST   STL  BLK   TOV    PF    PTS  +/-    TS%   eFG%  \
 0  12.0  37.0  49.0  30.0   5.0  6.0  18.0  25.0  122.0  NaN    0.6  0.565   
 1   9.0  30.0  39.0  20.0  12.0  7.0  13.0  19.0  126.0  NaN  0.617  0.542   
 
     3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%   USG%   ORtg  \
 0  0.272  0.239  28.6  80.4  55.7  66.7   4.8  13.6  15.0  100.0  117.2   
 1  0.476  0.488  19.6  71.4  44.3  52.6  11.5  10.4  11.3  100.0  121.0   
 
     DRtg   BPM MP_max FG_max FGA_max FG%_max 3P_max 3PA_max 3P%_max FT_max  \
 0  121.0  None    NaN   13.0    28.0   0.727    4.0     6.0     1.0    6.0   
 1  117.2  None    NaN   11.0    19.0   0.579    5.0    14.0     1.0   14.0   
 
   FTA_max FT%_max ORB_max DRB_max TRB_max AST_max STL_max 

-----------------------------------------------------------------------------
## -- 4.7 Unificar datos después de obtenerlos del html

##### Una vez corregidos todos los errores en los archivos, se procede a combinar todos los datos

In [68]:
# Combinar todos los partidos de los archivos NO corruptos

df_full_matchup = pd.concat(full_matchup, ignore_index = True)
pd.set_option('display.max_columns', None)
df_full_matchup

,MP,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,MP_max,FG_max,FGA_max,FG%_max,3P_max,3PA_max,3P%_max,FT_max,FTA_max,FT%_max,ORB_max,DRB_max,TRB_max,AST_max,STL_max,BLK_max,TOV_max,PF_max,PTS_max,+/-_max,TS%_max,eFG%_max,3PAr_max,FTr_max,ORB%_max,DRB%_max,TRB%_max,AST%_max,STL%_max,BLK%_max,TOV%_max,USG%_max,ORtg_max,DRtg_max,BPM_max,team,total,home,index_opponent,MP_opponent,MP_opponent,FG_opponent,FGA_opponent,FG%_opponent,3P_opponent,3PA_opponent,3P%_opponent,FT_opponent,FTA_opponent,FT%_opponent,ORB_opponent,DRB_opponent,TRB_opponent,AST_opponent,STL_opponent,BLK_opponent,TOV_opponent,PF_opponent,PTS_opponent,+/-_opponent,TS%_opponent,eFG%_opponent,3PAr_opponent,FTr_opponent,ORB%_opponent,DRB%_opponent,TRB%_opponent,AST%_opponent,STL%_opponent,BLK%_opponent,TOV%_opponent,USG%_opponent,ORtg_opponent,DRtg_opponent,BPM_opponent,MP_max_opponent,FG_max_opponent,FGA_max_opponent,FG%_max_opponent,3P_max_opponent,3PA_max_opponent,3P%_max_opponent,FT_max_opponent,FTA_max_opponent,FT%_max_opponent,ORB_max_opponent,DRB_max_opponent,TRB_max_opponent,AST_max_opponent,STL_max_opponent,BLK_max_opponent,TOV_max_opponent,PF_max_opponent,PTS_max_opponent,+/-_max_opponent,TS%_max_opponent,eFG%_max_opponent,3PAr_max_opponent,FTr_max_opponent,ORB%_max_opponent,DRB%_max_opponent,TRB%_max_opponent,AST%_max_opponent,STL%_max_opponent,BLK%_max_opponent,TOV%_max_opponent,USG%_max_opponent,ORtg_max_opponent,DRtg_max_opponent,BPM_max_opponent,team_opponent,total_opponent,home_opponent,season,date,winner
0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.600,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.800,3.0,7.0,1.000,3.0,5.0,1.000,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.900,0.900,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.570,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.750,3.0,6.0,0.667,5.0,7.0,1.0,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.800,6.000,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,2013,2013-10-29,False
1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.570,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.750,3.0,6.0,0.667,5.0,7.0,1.000,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.800,6.000,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.600,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.800,3.0,7.0,1.000,3.0,5.0,1.0,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.900,0.900,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,2013,2013-10-29,True
2,240.0,240.0,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,NaN,0.553,0.542,0.253,0.277,22.7,62.5,43.5,65.9,11.0,6.3,14.7,100.0,102.6,115.6,NaN,NaN,8.0,15.0,1.000,3.0,6.0,1.000,5.0,10.0,1.000,4.0,7.0,11.0,11.0,5.0,3.0,4.0,5.0,19.0,2.0,1.000,1.000,0.667,1.000,12.3,25.3,16.2,42.8,6.6,19.0,31.8,41.2,200.0,127.0,32.2,LAC,103,0,1,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0.312,0.301,37.5,77.3,56.5,54.8,8.0,9.7,15.3,100.0,115.6,102.6,NaN,NaN,8.0,13.0,0.750,3.0,6.0,1.000,6.0,8.0,1.0,7.0,12.0,13.0,6.0,1.0,2.0,5.0,5.0,22.0,19.0,1.000,1.000,0.750,0.800,38.8,53.5,27.7,33.6,3.7,12.1,50.0,29.6,210.0,110.0,15.5,LAL,116,1,2013,2013-10-29,False
3,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0

In [69]:
# Combinar todos los partidos de los archivos corruptos

df_full_matchup_corrupted = pd.concat(full_matchup_corrupted, ignore_index = True)
df_full_matchup_corrupted

,MP,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,MP_max,FG_max,FGA_max,FG%_max,3P_max,3PA_max,3P%_max,FT_max,FTA_max,FT%_max,ORB_max,DRB_max,TRB_max,AST_max,STL_max,BLK_max,TOV_max,PF_max,PTS_max,+/-_max,TS%_max,eFG%_max,3PAr_max,FTr_max,ORB%_max,DRB%_max,TRB%_max,AST%_max,STL%_max,BLK%_max,TOV%_max,USG%_max,ORtg_max,DRtg_max,BPM_max,team,total,home,index_opponent,MP_opponent,MP_opponent,FG_opponent,FGA_opponent,FG%_opponent,3P_opponent,3PA_opponent,3P%_opponent,FT_opponent,FTA_opponent,FT%_opponent,ORB_opponent,DRB_opponent,TRB_opponent,AST_opponent,STL_opponent,BLK_opponent,TOV_opponent,PF_opponent,PTS_opponent,+/-_opponent,TS%_opponent,eFG%_opponent,3PAr_opponent,FTr_opponent,ORB%_opponent,DRB%_opponent,TRB%_opponent,AST%_opponent,STL%_opponent,BLK%_opponent,TOV%_opponent,USG%_opponent,ORtg_opponent,DRtg_opponent,BPM_opponent,MP_max_opponent,FG_max_opponent,FGA_max_opponent,FG%_max_opponent,3P_max_opponent,3PA_max_opponent,3P%_max_opponent,FT_max_opponent,FTA_max_opponent,FT%_max_opponent,ORB_max_opponent,DRB_max_opponent,TRB_max_opponent,AST_max_opponent,STL_max_opponent,BLK_max_opponent,TOV_max_opponent,PF_max_opponent,PTS_max_opponent,+/-_max_opponent,TS%_max_opponent,eFG%_max_opponent,3PAr_max_opponent,FTr_max_opponent,ORB%_max_opponent,DRB%_max_opponent,TRB%_max_opponent,AST%_max_opponent,STL%_max_opponent,BLK%_max_opponent,TOV%_max_opponent,USG%_max_opponent,ORtg_max_opponent,DRtg_max_opponent,BPM_max_opponent,team_opponent,total_opponent,home_opponent,season,date,winner
0,240.0,240.0,45.0,92.0,0.489,14.0,25.0,0.56,18.0,22.0,0.818,12.0,37.0,49.0,30.0,5.0,6.0,18.0,25.0,122.0,NaN,0.6,0.565,0.272,0.239,28.6,80.4,55.7,66.7,4.8,13.6,15.0,100.0,117.2,121.0,None,NaN,13.0,28.0,0.727,4.0,6.0,1.0,6.0,8.0,1.0,4.0,13.0,17.0,9.0,2.0,2.0,8.0,6.0,35.0,21.0,0.909,0.909,0.5,0.667,13.4,39.9,27.2,41.9,6.4,15.7,48.2,37.9,164.0,127.0,None,MEM,122,0,1,240.0,240.0,38.0,84.0,0.452,15.0,40.0,0.375,35.0,41.0,0.854,9.0,30.0,39.0,20.0,12.0,7.0,13.0,19.0,126.0,NaN,0.617,0.542,0.476,0.488,19.6,71.4,44.3,52.6,11.5,10.4,11.3,100.0,121.0,117.2,None,NaN,11.0,19.0,0.579,5.0,14.0,1.0,14.0,14.0,1.0,6.0,15.0,21.0,10.0,5.0,2.0,5.0,5.0,31.0,18.0,0.75,0.75,1.0,0.933,15.4,42.2,28.2,33.8,6.7,10.4,22.5,35.6,159.0,128.0,None,POR,126,1,2020,2020-08-15,False
1,240.0,240.0,38.0,84.0,0.452,15.0,40.0,0.375,35.0,41.0,0.854,9.0,30.0,39.0,20.0,12.0,7.0,13.0,19.0,126.0,NaN,0.617,0.542,0.476,0.488,19.6,71.4,44.3,52.6,11.5,10.4,11.3,100.0,121.0,117.2,None,NaN,11.0,19.0,0.579,5.0,14.0,1.0,14.0,14.0,1.0,6.0,15.0,21.0,10.0,5.0,2.0,5.0,5.0,31.0,18.0,0.75,0.75,1.0,0.933,15.4,42.2,28.2,33.8,6.7,10.4,22.5,35.6,159.0,128.0,None,POR,126,1,0,240.0,240.0,45.0,92.0,0.489,14.0,25.0,0.56,18.0,22.0,0.818,12.0,37.0,49.0,30.0,5.0,6.0,18.0,25.0,122.0,NaN,0.6,0.565,0.272,0.239,28.6,80.4,55.7,66.7,4.8,13.6,15.0,100.0,117.2,121.0,None,NaN,13.0,28.0,0.727,4.0,6.0,1.0,6.0,8.0,1.0,4.0,13.0,17.0,9.0,2.0,2.0,8.0,6.0,35.0,21.0,0.909,0.909,0.5,0.667,13.4,39.9,27.2,41.9,6.4,15.7,48.2,37.9,164.0,127.0,None,MEM,122,0,2020,2020-08-15,True
2,240.0,240.0,40.0,92.0,0.435,3.0,21.0,0.143,17.0,20.0,0.85,13.0,36.0,49.0,17.0,5.0,9.0,15.0,31.0,100.0,NaN,0.496,0.451,0.228,0.217,27.1,72.0,50.0,42.5,5.0,17.6,13.0,100.0,99.3,117.2,None,NaN,10.0,25.0,1.0,1.0,7.0,1.0,8.0,8.0,1.0,5.0,9.0,14.0,6.0,3.0,2.0,4.0,5.0,22.0,6.0,1.0,1.0,1.0,0.5,51.7,25.5,25.3,62.1,3.9,9.2,50.0,35.2,200.0,128.0,None,WAS,100,0,1,240.0,240.0,38.0,96.0,0.396,15.0,45.0,0.333,27.0,32.0,0.844,14.0,35.0,49.0,20.0,8.0,5.0,10.0,17.0,118.0,NaN,0.536,0.474,0.469,0.333,28.0,72.9,50.0,52.6,7.9,7.0,8.3,100.0,117.2,99.3,None,NaN,14.0,32.0,1.0,6.0,14.0,0.429,17.0,17.0,1.0,6.0,6.0,12.0,6.0,2.0,2.0,2.0,4.0,50.0,25.0,1.064,1.0,1.0,2.0,49.7,28.1,25.3,30.3,3.5,5.0,20.0,41.4,239.0,109.0,None,BOS,118,1,2021,2021-05-18,False
3,240.0,240.0,38.0,96.0,0.396,15.0,45.0,0.333,27.0,32.0,0.844,14.0,35.0,49.0,20.0,8.0,5.0,10.0,17.0,118.0,NaN,0.536,0.474,0.469,0.333,28.0,72.9,50

##### Unificar ambos dataframes:

In [70]:
df_unified = pd.concat([df_full_matchup, df_full_matchup_corrupted], ignore_index=True)
df_unified

,MP,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,MP_max,FG_max,FGA_max,FG%_max,3P_max,3PA_max,3P%_max,FT_max,FTA_max,FT%_max,ORB_max,DRB_max,TRB_max,AST_max,STL_max,BLK_max,TOV_max,PF_max,PTS_max,+/-_max,TS%_max,eFG%_max,3PAr_max,FTr_max,ORB%_max,DRB%_max,TRB%_max,AST%_max,STL%_max,BLK%_max,TOV%_max,USG%_max,ORtg_max,DRtg_max,BPM_max,team,total,home,index_opponent,MP_opponent,MP_opponent,FG_opponent,FGA_opponent,FG%_opponent,3P_opponent,3PA_opponent,3P%_opponent,FT_opponent,FTA_opponent,FT%_opponent,ORB_opponent,DRB_opponent,TRB_opponent,AST_opponent,STL_opponent,BLK_opponent,TOV_opponent,PF_opponent,PTS_opponent,+/-_opponent,TS%_opponent,eFG%_opponent,3PAr_opponent,FTr_opponent,ORB%_opponent,DRB%_opponent,TRB%_opponent,AST%_opponent,STL%_opponent,BLK%_opponent,TOV%_opponent,USG%_opponent,ORtg_opponent,DRtg_opponent,BPM_opponent,MP_max_opponent,FG_max_opponent,FGA_max_opponent,FG%_max_opponent,3P_max_opponent,3PA_max_opponent,3P%_max_opponent,FT_max_opponent,FTA_max_opponent,FT%_max_opponent,ORB_max_opponent,DRB_max_opponent,TRB_max_opponent,AST_max_opponent,STL_max_opponent,BLK_max_opponent,TOV_max_opponent,PF_max_opponent,PTS_max_opponent,+/-_max_opponent,TS%_max_opponent,eFG%_max_opponent,3PAr_max_opponent,FTr_max_opponent,ORB%_max_opponent,DRB%_max_opponent,TRB%_max_opponent,AST%_max_opponent,STL%_max_opponent,BLK%_max_opponent,TOV%_max_opponent,USG%_max_opponent,ORtg_max_opponent,DRtg_max_opponent,BPM_max_opponent,team_opponent,total_opponent,home_opponent,season,date,winner
0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.6,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.8,3.0,7.0,1.0,3.0,5.0,1.0,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.9,0.9,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.57,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.75,3.0,6.0,0.667,5.0,7.0,1.0,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.8,6.0,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,2013,2013-10-29,False
1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.57,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.75,3.0,6.0,0.667,5.0,7.0,1.0,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.8,6.0,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.6,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.8,3.0,7.0,1.0,3.0,5.0,1.0,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.9,0.9,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,2013,2013-10-29,True
2,240.0,240.0,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,NaN,0.553,0.542,0.253,0.277,22.7,62.5,43.5,65.9,11.0,6.3,14.7,100.0,102.6,115.6,NaN,NaN,8.0,15.0,1.0,3.0,6.0,1.0,5.0,10.0,1.0,4.0,7.0,11.0,11.0,5.0,3.0,4.0,5.0,19.0,2.0,1.0,1.0,0.667,1.0,12.3,25.3,16.2,42.8,6.6,19.0,31.8,41.2,200.0,127.0,32.2,LAC,103,0,1,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0.312,0.301,37.5,77.3,56.5,54.8,8.0,9.7,15.3,100.0,115.6,102.6,NaN,NaN,8.0,13.0,0.75,3.0,6.0,1.0,6.0,8.0,1.0,7.0,12.0,13.0,6.0,1.0,2.0,5.0,5.0,22.0,19.0,1.0,1.0,0.75,0.8,38.8,53.5,27.7,33.6,3.7,12.1,50.0,29.6,210.0,110.0,15.5,LAL,116,1,2013,2013-10-29,False
3,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0.312,0.301,37.5,77.3,56.5,54.8,8.0,9.7,15.3,100.0,115.6,10

In [71]:
# Aquí se puede observar que dice el ganador ("winner") para la variable "team"

df_unified.loc[0:5 , ["team", "total", "team_opponent", "total_opponent", "winner", "date"]]

,team,total,team_opponent,total_opponent,winner,date
0,ORL,87,IND,97,False,2013-10-29
1,IND,97,ORL,87,True,2013-10-29
2,LAC,103,LAL,116,False,2013-10-29
3,LAL,116,LAC,103,True,2013-10-29
4,CHI,95,MIA,107,False,2013-10-29
5,MIA,107,CHI,95,True,2013-10-29


In [72]:
# Comprobar las filas que no tienen el mismo número de columnas (puede observar si alguna variable tiene + o - variables)


for idx, row in df_unified.iterrows():
    if len(row) != 152:
        print(f"La fila {idx} tiene {len(row)} columnas, no 152.")

#### Guardar el dataframe

Se obtiene una base de datos con 25.672 filas y 152 columnas

In [73]:
df_unified.to_csv("nba_stats.csv", index=False)

In [74]:
pd.set_option('display.max_columns', None)
df_unified

,MP,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,MP_max,FG_max,FGA_max,FG%_max,3P_max,3PA_max,3P%_max,FT_max,FTA_max,FT%_max,ORB_max,DRB_max,TRB_max,AST_max,STL_max,BLK_max,TOV_max,PF_max,PTS_max,+/-_max,TS%_max,eFG%_max,3PAr_max,FTr_max,ORB%_max,DRB%_max,TRB%_max,AST%_max,STL%_max,BLK%_max,TOV%_max,USG%_max,ORtg_max,DRtg_max,BPM_max,team,total,home,index_opponent,MP_opponent,MP_opponent,FG_opponent,FGA_opponent,FG%_opponent,3P_opponent,3PA_opponent,3P%_opponent,FT_opponent,FTA_opponent,FT%_opponent,ORB_opponent,DRB_opponent,TRB_opponent,AST_opponent,STL_opponent,BLK_opponent,TOV_opponent,PF_opponent,PTS_opponent,+/-_opponent,TS%_opponent,eFG%_opponent,3PAr_opponent,FTr_opponent,ORB%_opponent,DRB%_opponent,TRB%_opponent,AST%_opponent,STL%_opponent,BLK%_opponent,TOV%_opponent,USG%_opponent,ORtg_opponent,DRtg_opponent,BPM_opponent,MP_max_opponent,FG_max_opponent,FGA_max_opponent,FG%_max_opponent,3P_max_opponent,3PA_max_opponent,3P%_max_opponent,FT_max_opponent,FTA_max_opponent,FT%_max_opponent,ORB_max_opponent,DRB_max_opponent,TRB_max_opponent,AST_max_opponent,STL_max_opponent,BLK_max_opponent,TOV_max_opponent,PF_max_opponent,PTS_max_opponent,+/-_max_opponent,TS%_max_opponent,eFG%_max_opponent,3PAr_max_opponent,FTr_max_opponent,ORB%_max_opponent,DRB%_max_opponent,TRB%_max_opponent,AST%_max_opponent,STL%_max_opponent,BLK%_max_opponent,TOV%_max_opponent,USG%_max_opponent,ORtg_max_opponent,DRtg_max_opponent,BPM_max_opponent,team_opponent,total_opponent,home_opponent,season,date,winner
0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.6,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.8,3.0,7.0,1.0,3.0,5.0,1.0,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.9,0.9,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.57,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.75,3.0,6.0,0.667,5.0,7.0,1.0,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.8,6.0,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,2013,2013-10-29,False
1,240.0,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,20.0,13.0,97.0,NaN,0.57,0.528,0.239,0.451,27.8,72.3,53.0,50.0,4.2,24.3,19.0,100.0,102.1,91.6,NaN,NaN,8.0,16.0,0.75,3.0,6.0,0.667,5.0,7.0,1.0,6.0,10.0,16.0,5.0,2.0,7.0,5.0,5.0,24.0,15.0,0.922,0.875,0.8,6.0,29.7,38.0,34.4,29.0,3.3,16.9,50.0,29.2,150.0,104.0,10.1,IND,97,1,0,240.0,240.0,36.0,93.0,0.387,9.0,19.0,0.474,6.0,10.0,0.6,13.0,26.0,39.0,17.0,10.0,6.0,17.0,26.0,87.0,NaN,0.447,0.435,0.204,0.108,27.7,72.2,47.0,47.2,10.5,11.1,14.9,100.0,91.6,102.1,NaN,NaN,8.0,14.0,0.8,3.0,7.0,1.0,3.0,5.0,1.0,5.0,5.0,10.0,7.0,2.0,3.0,5.0,4.0,18.0,10.0,0.9,0.9,0.538,0.455,19.0,37.1,21.6,54.9,3.9,18.5,31.3,30.1,151.0,111.0,17.3,ORL,87,0,2013,2013-10-29,True
2,240.0,240.0,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,NaN,0.553,0.542,0.253,0.277,22.7,62.5,43.5,65.9,11.0,6.3,14.7,100.0,102.6,115.6,NaN,NaN,8.0,15.0,1.0,3.0,6.0,1.0,5.0,10.0,1.0,4.0,7.0,11.0,11.0,5.0,3.0,4.0,5.0,19.0,2.0,1.0,1.0,0.667,1.0,12.3,25.3,16.2,42.8,6.6,19.0,31.8,41.2,200.0,127.0,32.2,LAC,103,0,1,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0.312,0.301,37.5,77.3,56.5,54.8,8.0,9.7,15.3,100.0,115.6,102.6,NaN,NaN,8.0,13.0,0.75,3.0,6.0,1.0,6.0,8.0,1.0,7.0,12.0,13.0,6.0,1.0,2.0,5.0,5.0,22.0,19.0,1.0,1.0,0.75,0.8,38.8,53.5,27.7,33.6,3.7,12.1,50.0,29.6,210.0,110.0,15.5,LAL,116,1,2013,2013-10-29,False
3,240.0,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,NaN,0.551,0.527,0.312,0.301,37.5,77.3,56.5,54.8,8.0,9.7,15.3,100.0,115.6,10